<a href="https://colab.research.google.com/github/RebecaGis/LiDAR_CHM/blob/main/Aplicativo_Shiny_para_Processamento_LiDAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#
# Aplicativo Shiny para Processamento LiDAR - Versão Simplificada com Progresso
#

# Verificar e instalar pacotes necessários
required_packages <- c("shiny", "shinythemes", "shinyFiles", "fs", "lidR", "sf", "terra", "DT")
missing_packages <- required_packages[!required_packages %in% installed.packages()[,"Package"]]

if (length(missing_packages) > 0) {
  message("Instalando pacotes necessários: ", paste(missing_packages, collapse = ", "))
  install.packages(missing_packages)
}

# Carregar pacotes
library(shiny)
library(shinythemes)
library(shinyFiles)
library(fs)
library(sf)
library(terra)
library(DT)

# Verificar se o lidR está disponível
lidR_available <- require("lidR", quietly = TRUE)
if (!lidR_available) {
  message("Pacote lidR não está disponível. Algumas funcionalidades estarão limitadas.")
}

# Configurações para melhor performance
options(lidR.progress = FALSE)
if (lidR_available) {
  options(lidR.check.nested.parallelism = FALSE)
  lidR:::set_lidr_threads(1)  # Reduzir para 1 thread para evitar sobrecarga
}

# Desativar S2 para melhor performance em operações geográficas
sf::sf_use_s2(FALSE)

# FUNÇÃO GERAR DTM (ADICIONADA)
gerar_dtm <- function(las, res = 1) {
  if (!lidR_available) stop("lidR não disponível para gerar DTM")
  if (lidR::is.empty(las) || lidR::npoints(las) == 0) {
    message("Arquivo LAS vazio - pulando geração de DTM")
    return(NULL)
  }

  # Verificar se há pontos de solo
  ground_points <- sum(las$Classification == 2, na.rm = TRUE)
  if (ground_points == 0) {
    message("Nenhum ponto de solo encontrado - classificando...")
    las <- classificar_solo_csf(las)
    ground_points <- sum(las$Classification == 2, na.rm = TRUE)
    if (ground_points == 0) {
      message("Ainda não há pontos de solo após classificação - retornando NULL")
      return(NULL)
    }
  }

  # Gerar DTM apenas com pontos de solo
  dtm <- lidR::rasterize_terrain(las, res = res, algorithm = lidR::tin())
  return(dtm)
}

classificar_solo_csf <- function(las) {
  if (!lidR_available) stop("lidR não disponível para classificação do solo")
  if (lidR::is.empty(las) || lidR::npoints(las) == 0) {
    message("Arquivo LAS vazio - pulando classificação do solo")
    return(las)
  }

  csf_algo <- lidR::csf(
    cloth_resolution = 0.5,
    class_threshold = 0.5,
    time_step = 0.65,
    sloop_smooth = FALSE
  )
  lidR::classify_ground(las, algorithm = csf_algo)
}

gerar_dsm <- function(las, res, suavizar = TRUE, clip_percentil = NA) {
  if (!lidR_available) stop("lidR não disponível para gerar DSM")
  pf_algo <- lidR::pitfree(thresholds = c(0, 2, 5, 10, 15, 20), subcircle = 0.2)
  dsm <- lidR::rasterize_canopy(las, res = res, algorithm = pf_algo)

  if (suavizar) {
    dsm <- terra::focal(dsm, w = matrix(1, 3, 3), fun = median, na.rm = TRUE)
  }

  if (!is.na(clip_percentil)) {
    vals <- terra::values(dsm, na.rm = TRUE)
    if (length(vals) > 0) {
      lim <- quantile(vals, probs = clip_percentil/100, na.rm = TRUE)
      dsm <- terra::clamp(dsm, lower = min(vals, na.rm = TRUE), upper = lim, values = TRUE)
    }
  }

  dsm
}

alinhar_para <- function(src, template) {
  if (!terra::compareGeom(template, src, stopOnError = FALSE)) {
    terra::resample(src, template, method = "bilinear")
  } else {
    src
  }
}

estatisticas <- function(r) {
  v <- terra::values(r, na.rm = TRUE)
  if (length(v) == 0) {
    return(list(min = NA, max = NA, mean = NA, median = NA, p1 = NA, p50 = NA, p99 = NA))
  }
  list(
    min = min(v, na.rm = TRUE),
    max = max(v, na.rm = TRUE),
    mean = mean(v, na.rm = TRUE),
    median = median(v, na.rm = TRUE),
    p1 = quantile(v, 0.01, na.rm = TRUE),
    p50 = quantile(v, 0.50, na.rm = TRUE),
    p99 = quantile(v, 0.99, na.rm = TRUE)
  )
}

tratar_ruido <- function(las) {
  if (!lidR_available) return(las)

  # Verificar se há pontos antes de processar
  if (lidR::is.empty(las)) return(las)

  # Remover ruídos
  las <- lidR::classify_noise(las, algorithm = lidR::sor(k = 8, m = 2))
  las <- lidR::filter_poi(las, Classification != LASNOISE)

  return(las)
}

# Função para verificar e corrigir compatibilidade de projeção
verificar_projecao <- function(las, shapefile) {
  tryCatch({
    # Obter CRS do LAS
    las_crs <- sf::st_crs(las)

    # Obter CRS do shapefile
    shp_crs <- sf::st_crs(shapefile)

    # Se os CRS forem diferentes, transformar o shapefile para o CRS do LAS
    if (!is.na(las_crs$epsg) && !is.na(shp_crs$epsg) && las_crs != shp_crs) {
      message(paste("Transformando shapefile de", shp_crs$epsg, "para", las_crs$epsg))
      shapefile <- sf::st_transform(shapefile, crs = las_crs)
    }

    return(shapefile)
  }, error = function(e) {
    message(paste("Erro na verificação de projeção:", e$message))
    return(shapefile)
  })
}

# Função para verificar interseção prévia entre LAS e shapefile (versão corrigida)
verificar_intersecoes <- function(input_dir, buffer_shp, session) {
  tryCatch({
    # Ler o shapefile
    uts <- sf::st_read(buffer_shp, quiet = TRUE)

    # Listar arquivos LAS
    las_files <- fs::dir_ls(input_dir, regexp = "\\.las$", recurse = FALSE)
    n_files <- length(las_files)

    if (n_files == 0) {
      return(list(intersecoes = NULL, mensagem = "Nenhum arquivo LAS encontrado"))
    }

    intersecoes <- list()

    withProgress(message = 'Verificando interseções', value = 0, {
      for (i in seq_along(las_files)) {
        las_file <- las_files[i]
        filename <- fs::path_file(las_file)
        base_name <- fs::path_ext_remove(filename)

        incProgress(1/n_files, detail = paste("Verificando:", base_name))

        tryCatch({
          # Ler apenas o cabeçalho do LAS para obter a extensão
          las_header <- lidR::readLASheader(las_file)

          # Obter CRS do LAS
          las_crs <- sf::st_crs(las_header)

          # Obter CRS do shapefile
          shp_crs <- sf::st_crs(uts)

          # Se os CRS forem diferentes, transformar o shapefile para o CRS do LAS
          if (!is.na(las_crs$epsg) && !is.na(shp_crs$epsg) && las_crs != shp_crs) {
            message(paste("Transformando shapefile de", shp_crs$epsg, "para", las_crs$epsg))
            uts_transformed <- sf::st_transform(uts, crs = las_crs)
          } else {
            uts_transformed <- uts
          }

          # Criar bbox do LAS
          las_bbox <- sf::st_bbox(c(
            xmin = las_header@PHB[["Min X"]],
            ymin = las_header@PHB[["Min Y"]],
            xmax = las_header@PHB[["Max X"]],
            ymax = las_header@PHB[["Max Y"]]
          ), crs = las_crs)

          # Converter para sf object
          las_polygon <- sf::st_as_sfc(las_bbox)

          # Verificar interseção com cada polígono do shapefile
          for (j in seq_len(nrow(uts_transformed))) {
            polygon <- uts_transformed[j, ]

            # Verificar se há interseção
            intersects <- sf::st_intersects(las_polygon, polygon, sparse = FALSE)[1, 1]

            if (intersects) {
              # Adicionar à lista de interseções
              intersecoes[[length(intersecoes) + 1]] <- list(
                las_file = las_file,
                polygon_index = j,
                polygon_id = ifelse("Field" %in% names(uts), uts$Field[j], j),
                polygon_geom = polygon,
                original_polygon_geom = uts[j, ]  # Manter a geometria original também
              )
              message(paste("Interseção encontrada:", base_name, "com polígono", j))
            }
          }

        }, error = function(e) {
          message(paste("Erro ao verificar interseção para", base_name, ":", e$message))
        })
      }
    })

    return(list(
      intersecoes = intersecoes,
      mensagem = paste("Verificação concluída. Encontradas", length(intersecoes), "interseções")
    ))

  }, error = function(e) {
    return(list(intersecoes = NULL, mensagem = paste("Erro na verificação de interseções:", e$message)))
  })
}

# Função para processar corte por buffer com verificação prévia (versão corrigida)
processar_corte_buffer <- function(input_dir, output_dir, buffer_shp, session) {
  tryCatch({
    # Primeiro, verificar todas as interseções
    verificacao <- verificar_intersecoes(input_dir, buffer_shp, session)

    if (is.null(verificacao$intersecoes) || length(verificacao$intersecoes) == 0) {
      return(verificacao$mensagem)
    }

    intersecoes <- verificacao$intersecoes
    n_intersecoes <- length(intersecoes)

    arquivos_processados <- 0
    features_processadas <- 0
    arquivos_ignorados <- 0

    withProgress(message = 'Corte por Shapefile', value = 0, {
      for (i in seq_along(intersecoes)) {
        interseccao <- intersecoes[[i]]

        las_file <- interseccao$las_file
        filename <- fs::path_file(las_file)
        base_name <- fs::path_ext_remove(filename)
        polygon_index <- interseccao$polygon_index
        polygon_id <- interseccao$polygon_id

        incProgress(1/n_intersecoes, detail = paste("Processando:", base_name, "- Polígono", polygon_id))

        tryCatch({
          # Carregar o arquivo LAS
          las <- lidR::readLAS(las_file)
          if (lidR::is.empty(las)) {
            message(paste("Arquivo", base_name, "está vazio - ignorando"))
            arquivos_ignorados <- arquivos_ignorados + 1
            next
          }

          # Usar a geometria já transformada da verificação
          uts_corrigido <- interseccao$polygon_geom

          # Nome do arquivo de saída
          output_las <- fs::path(output_dir, paste0(base_name, "_", polygon_id, ".las"))

          # CORREÇÃO: Remover dimensão Z antes de converter para Spatial
          uts_corrigido_2d <- sf::st_zm(uts_corrigido)  # Remove dimensão Z

          # Converter para SpatialPolygonsDataFrame (formato compatível com lidR)
          uts_sp <- as(uts_corrigido_2d, "Spatial")

          # Cortar usando clip_roi
          las_crop <- lidR::clip_roi(las, uts_sp)

          # Alternativa: se ainda não funcionar, tentar método manual
          if (is.null(las_crop) || lidR::is.empty(las_crop)) {
            message(paste("Tentando método manual para polígono", polygon_id))

            # Método manual: filtrar pontos dentro do polígono
            coords <- cbind(las$X, las$Y)
            points_sf <- sf::st_as_sf(data.frame(coords), coords = c("X1", "X2"), crs = sf::st_crs(las))

            # Verificar quais pontos estão dentro do polígono
            inside <- sf::st_intersects(points_sf, uts_corrigido_2d, sparse = FALSE)
            inside <- as.logical(inside[,1])

            if (sum(inside, na.rm = TRUE) > 0) {
              # Criar LAS manualmente com pontos dentro do polígono
              las_crop <- las[inside]
            }
          }

          if (is.null(las_crop) || lidR::is.empty(las_crop)) {
            message(paste("Nenhum ponto intercepta o polígono", polygon_id, "no arquivo", base_name))
            next
          }

          # Verificar se há pontos após o corte
          if (lidR::npoints(las_crop) == 0) {
            message(paste("Corte realizado mas sem pontos no polígono", polygon_id, "do arquivo", base_name))
            next
          }

          # Escrever arquivo LAS cortado
          lidR::writeLAS(las_crop, output_las)
          features_processadas <- features_processadas + 1
          message(paste("Polígono", polygon_id, "do arquivo", base_name,
                        "salvo com", lidR::npoints(las_crop), "pontos"))

          # Contar como arquivo processado se pelo menos uma feature foi processada
          arquivos_processados <- length(unique(sapply(intersecoes[1:i], function(x) x$las_file)))

          # Limpar memória
          rm(las, las_crop)
          gc()

        }, error = function(e) {
          message(paste("Erro ao processar arquivo", base_name, "com polígono", polygon_id, ":", e$message))
          arquivos_ignorados <- arquivos_ignorados + 1
        })
      }
    })

    return(paste("Corte por shapefile concluído! Arquivos processados:", arquivos_processados,
                 "| Features criadas:", features_processadas,
                 "| Interseções ignoradas:", arquivos_ignorados))

  }, error = function(e) {
    return(paste("Erro no corte por shapefile:", e$message))
  })
}

# Funções de processamento com progresso
processar_georreferenciamento <- function(input_dir, output_dir, session) {
  tryCatch({
    if (!lidR_available) return("lidR não disponível para georreferenciamento")

    las_files <- fs::dir_ls(input_dir, regexp = "\\.las$", recurse = FALSE)
    n_files <- length(las_files)
    if (n_files == 0) return("Nenhum arquivo LAS encontrado na pasta de entrada.")

    withProgress(message = 'Georreferenciamento', value = 0, {
      for (i in seq_along(las_files)) {
        las_file <- las_files[i]
        filename <- fs::path_file(las_file)
        output_las <- fs::path(output_dir, gsub("\\.las$", "_ref.las", filename))
        incProgress(1/n_files, detail = paste("Processando", filename))

        tryCatch({
          las <- lidR::readLAS(las_file)
          if (!lidR::is.empty(las)) {
            sf::st_crs(las) <- 31983
            lidR::writeLAS(las, output_las)
          }
        }, error = function(e) {
          message(paste("Erro no georreferenciamento de", filename, ":", e$message))
        })
      }
    })

    return("Georreferenciamento concluído com sucesso!")
  }, error = function(e) {
    return(paste("Erro no georreferenciamento:", e$message))
  })
}

processar_chm <- function(input_dir, output_dir_dtm_dsm, output_dir_chm, resolucao,
                          suavizar_dsm, percentil_clip, session) {
  tryCatch({
    if (!lidR_available) return(list(status = "lidR não disponível para processamento CHM", relatorio = NULL))

    las_files <- fs::dir_ls(input_dir, glob = "*.las")
    n_files <- length(las_files)
    if (n_files == 0) return(list(status = "Nenhum arquivo LAS encontrado para processar CHM.", relatorio = NULL))

    relatorio_csv <- file.path(output_dir_dtm_dsm, "CHM_stats_report.csv")
    write.csv(data.frame(UT = character(), Arquivo_CHM = character(),
                         Min = numeric(), Max = numeric(), Media = numeric(), Mediana = numeric(),
                         P1 = numeric(), P50 = numeric(), P99 = numeric()),
              relatorio_csv, row.names = FALSE)

    arquivos_processados <- 0
    arquivos_ignorados <- 0

    withProgress(message = 'Geração de CHM', value = 0, {
      for (i in seq_along(las_files)) {
        las_file <- las_files[i]
        base <- fs::path_ext_remove(fs::path_file(las_file))
        incProgress(1/n_files, detail = paste("Processando", base))

        tryCatch({
          las <- lidR::readLAS(las_file)
          if (is.null(las) || lidR::is.empty(las)) {
            message(paste("Arquivo", base, "está vazio - ignorando"))
            arquivos_ignorados <- arquivos_ignorados + 1
            next
          }

          if (lidR::npoints(las) == 0) {
            message(paste("Arquivo", base, "não tem pontos - ignorando"))
            arquivos_ignorados <- arquivos_ignorados + 1
            next
          }

          # Verificar se há pontos de solo antes de classificar
          if (sum(las$Classification == 2, na.rm = TRUE) == 0) {
            message(paste("Arquivo", base, "não tem pontos de solo classificados - classificando..."))
            las <- classificar_solo_csf(las)
          }

          lidR::projection(las) <- "EPSG:31983"
          las <- tratar_ruido(las)

          if (lidR::npoints(las) == 0) {
            message(paste("Arquivo", base, "ficou vazio após remoção de ruído - ignorando"))
            arquivos_ignorados <- arquivos_ignorados + 1
            next
          }

          # Gerar DTM usando a nova função
          dtm <- gerar_dtm(las, res = resolucao)

          if (is.null(dtm)) {
            message(paste("Arquivo", base, "não foi possível gerar DTM - ignorando"))
            arquivos_ignorados <- arquivos_ignorados + 1
            next
          }

          # Gerar DSM
          dsm <- gerar_dsm(las, res = resolucao, suavizar = suavizar_dsm, clip_percentil = percentil_clip)
          dsm <- alinhar_para(dsm, dtm)

          dtm_tif <- file.path(output_dir_dtm_dsm, paste0(base, "_DTM.tif"))
          dsm_tif <- file.path(output_dir_dtm_dsm, paste0(base, "_DSM.tif"))
          chm_tif <- file.path(output_dir_chm, paste0(base, "_CHM.tif"))

          terra::writeRaster(dtm, dtm_tif, overwrite = TRUE, gdal = c("COMPRESS=LZW"))
          terra::writeRaster(dsm, dsm_tif, overwrite = TRUE, gdal = c("COMPRESS=LZW"))

          chm <- dsm - dtm
          chm[chm < 0] <- 0
          chm[is.na(chm)] <- 0
          terra::writeRaster(chm, chm_tif, overwrite = TRUE, gdal = c("COMPRESS=LZW"))

          stats <- estatisticas(chm)
          write.table(data.frame(
            UT = base, Arquivo_CHM = chm_tif,
            Min = stats$min, Max = stats$max, Media = stats$mean, Mediana = stats$median,
            P1 = stats$p1, P50 = stats$p50, P99 = stats$p99
          ), relatorio_csv, sep = ",", col.names = FALSE, row.names = FALSE, append = TRUE)

          arquivos_processados <- arquivos_processados + 1

        }, error = function(e) {
          message(paste("Erro ao processar", base, ":", e$message))
          arquivos_ignorados <- arquivos_ignorados + 1
        })
      }
    })

    return(list(
      status = paste("Processamento DTM/DSM/CHM concluído! Processados:", arquivos_processados,
                     "| Ignorados:", arquivos_ignorados),
      relatorio = if (arquivos_processados > 0) relatorio_csv else NULL
    ))
  }, error = function(e) {
    return(list(status = paste("Erro no processamento CHM:", e$message), relatorio = NULL))
  })
}

# Interface do usuário
ui <- fluidPage(
  theme = shinytheme("flatly"),
  titlePanel("Processamento LiDAR - Fluxo Simplificado"),

  sidebarLayout(
    sidebarPanel(
      width = 4,
      h4("Configurações do Processamento"),

      wellPanel(
        h5("1. Configuração de Diretórios"),
        shinyDirButton("base_dir", "Pasta Base para Processamento", "Selecionar pasta base"),
        verbatimTextOutput("base_dir_text"),
        br(),
        h5("2. Shapefile de Recorte"),
        shinyFilesButton("buffer_shp_file", "Selecionar Shapefile", "Selecionar", multiple = FALSE,
                         filetypes = c('shp')),
        verbatimTextOutput("buffer_shp_text"),
        tags$div(style = "font-size: 12px; color: #666;",
                 "Nota: Cada polígono do shapefile gerará um arquivo LAS com nome original + identificador")
      ),

      wellPanel(
        h5("3. Parâmetros de Processamento"),
        numericInput("resolucao", "Resolução (metros):", value = 0.40, min = 0.1, max = 5, step = 0.1),
        checkboxInput("suavizar_dsm", "Aplicar suavização no DSM", value = TRUE),
        numericInput("percentil_clip", "Percentil para clip do DSM:", value = 99.5, min = 90, max = 100, step = 0.1)
      ),

      actionButton("processar", "Executar Processamento", class = "btn-success", style = "width: 100%;"),
      br(), br(),

      wellPanel(
        h5("Progresso do Processamento"),
        verbatimTextOutput("status_geral"),
        br(),
        h5("Progresso da Etapa Atual"),
        verbatimTextOutput("status_detalhado")
      )
    ),

    mainPanel(
      width = 8,
      tabsetPanel(
        tabPanel("Instruções",
                 h4("Como usar o aplicativo:"),
                 tags$ol(
                   tags$li("Selecione a pasta base onde serão criadas as subpastas"),
                   tags$li("Selecione o shapefile para recorte (cada polígono gera um arquivo LAS)"),
                   tags$li("Ajuste os parâmetros de processamento"),
                   tags$li("Clique em 'Executar Processamento'"),
                   tags$li("Acompanhe o progresso na área de status")
                 ),
                 tags$div(style = "background-color: #f8f9fa; padding: 15px; border-radius: 5px;",
                          h5("Informações importantes:"),
                          tags$ul(
                            tags$li("O sistema fará verificação prévia de interseções entre LAS e polígonos"),
                            tags$li("Apenas combinações com interseção serão processadas"),
                            tags$li("Os arquivos cortados terão nome: arquivoLAS_identificadorPoligono.las"),
                            tags$li("Se o shapefile tiver campo 'Field', será usado como identificador")
                          )
                 )
        ),
        tabPanel("Relatório", h4("Estatísticas dos CHM Gerados"), DTOutput("relatorio_table")),
        tabPanel("Log Detalhado", verbatimTextOutput("console_output"))
      )
    )
  )
)

# Server
server <- function(input, output, session) {
  volumes <- c(Home = fs::path_home(), "R Installation" = R.home(), getVolumes()())

  shinyDirChoose(input, "base_dir", roots = volumes, session = session)
  shinyFileChoose(input, "buffer_shp_file", roots = volumes, session = session, filetypes = c('shp'))

  paths <- reactiveValues(
    base_dir = "", input_dir = "", output_dir1 = "", output_dir2 = "",
    output_dir3 = "", output_dir4 = "", buffer_shp = ""
  )

  status <- reactiveValues(
    geral = "Aguardando início do processamento",
    detalhado = "Nenhuma etapa em execução",
    relatorio = NULL
  )

  observeEvent(input$base_dir, {
    if (!is.integer(input$base_dir)) {
      base_path <- parseDirPath(volumes, input$base_dir)
      paths$base_dir <- base_path
      paths$input_dir <- file.path(base_path, "Las_Originals")
      paths$output_dir1 <- file.path(base_path, "1_Las_Georeferencing")
      paths$output_dir2 <- file.path(base_path, "2_Las_Buffer_Crop")
      paths$output_dir3 <- file.path(base_path, "3_DTM_DSM")
      paths$output_dir4 <- file.path(base_path, "4_CHM")
    }
  })

  observeEvent(input$buffer_shp_file, {
    if (!is.integer(input$buffer_shp_file)) {
      paths$buffer_shp <- parseFilePaths(volumes, input$buffer_shp_file)$datapath
    }
  })

  output$base_dir_text <- renderText({ if (paths$base_dir != "") paths$base_dir else "Nenhuma pasta base selecionada" })
  output$buffer_shp_text <- renderText({ if (paths$buffer_shp != "") paths$buffer_shp else "Nenhum shapefile selecionado" })
  output$status_geral <- renderText({ status$geral })
  output$status_detalhado <- renderText({ status$detalhado })
  output$console_output <- renderText({ paste(status$geral, "\n", status$detalhado) })

  observeEvent(input$processar, {
    if (paths$base_dir == "") {
      status$geral <- "Erro: Selecione uma pasta base primeiro."
      return()
    }
    if (paths$buffer_shp == "") {
      status$geral <- "Erro: Selecione o shapefile primeiro."
      return()
    }

    # Verificar se o shapefile é válido
    if (!file.exists(paths$buffer_shp)) {
      status$geral <- "Erro: O arquivo shapefile não existe."
      return()
    }

    # Verificar se é um shapefile (extensão .shp)
    if (!grepl("\\.shp$", paths$buffer_shp, ignore.case = TRUE)) {
      status$geral <- "Erro: O arquivo selecionado não é um shapefile (.shp)."
      return()
    }

    # Verificar se lidR está disponível
    if (!lidR_available) {
      status$geral <- "Erro: O pacote lidR não está disponível. Instale-o com: install.packages('lidR')"
      return()
    }

    # Verificar se o shapefile tem features
    uts <- try(sf::st_read(paths$buffer_shp, quiet = TRUE), silent = TRUE)
    if (inherits(uts, "try-error") || nrow(uts) == 0) {
      status$geral <- "Erro: O shapefile não contém features/polígonos válidos."
      return()
    }

    dir.create(paths$output_dir1, showWarnings = FALSE, recursive = TRUE)
    dir.create(paths$output_dir2, showWarnings = FALSE, recursive = TRUE)
    dir.create(paths$output_dir3, showWarnings = FALSE, recursive = TRUE)
    dir.create(paths$output_dir4, showWarnings = FALSE, recursive = TRUE)

    # 1. Georreferenciamento
    status$geral <- "Iniciando georreferenciamento..."
    status$detalhado <- "Preparando arquivos..."
    resultado_geo <- processar_georreferenciamento(paths$input_dir, paths$output_dir1, session)
    status$geral <- resultado_geo
    status$detalhado <- "Georreferenciamento concluído"
    if (grepl("Erro", resultado_geo)) return()

    # 2. Corte por shapefile
    status$geral <- "Processando corte por shapefile..."
    status$detalhado <- "Verificando interseções..."
    resultado_buffer <- processar_corte_buffer(paths$output_dir1, paths$output_dir2, paths$buffer_shp, session)
    status$geral <- resultado_buffer
    status$detalhado <- "Corte por shapefile concluído"
    if (grepl("Erro", resultado_buffer)) return()

    # 3. Geração de DTM/DSM/CHM
    status$geral <- "Gerando DTM/DSM/CHM..."
    status$detalhado <- "Iniciando processamento de CHM..."
    resultado_chm <- processar_chm(paths$output_dir2, paths$output_dir3, paths$output_dir4,
                                   input$resolucao, input$suavizar_dsm, input$percentil_clip, session)
    status$geral <- resultado_chm$status
    status$detalhado <- "Processamento de DTM/DSM/CHM concluído"

    if (!is.null(resultado_chm$relatorio) && file.exists(resultado_chm$relatorio)) {
      status$relatorio <- read.csv(resultado_chm$relatorio)
    }
  })

  output$relatorio_table <- renderDT({
    req(status$relatorio)
    datatable(status$relatorio, options = list(pageLength = 10, scrollX = TRUE))
  })
}

shinyApp(ui, server)